In [1]:
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

def cleaning(text):
    # clean the text from URLs, hashtags, and stopwords
    
    # clear the text from punctuation
    punct = string.punctuation.replace('\'', '')
    text = [word.translate("".maketrans(punct, " "*len(punct))) for word in text.split()]

    # lemmatize text
    stemmer = WordNetLemmatizer()
    text = [stemmer.lemmatize(word).lower() for word in text]
    
    sw = stopwords.words('english')
    text = [word for word in text if word not in sw]
    
    # remove unecessary spaces
    text = re.sub('\s+', ' ', ' '.join(text).strip())
    
    return text

In [2]:
with open('./data/moliere_complete.txt', 'r') as f:
    moliere = ''.join(list(f))
    clean_moliere = cleaning(moliere)
    
dataset_moliere = [clean_moliere[i*100:(i+1)*100] for i in range(int(len(clean_moliere)/100))][:3000]

In [3]:
# load shakespeare db
import tensorflow as tf

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
shakespeare = open(path_to_file, 'rb').read().decode(encoding='utf-8')
clean_shakespeare= cleaning(shakespeare)
dataset_shakespeare = [clean_shakespeare[i*100:(i+1)*100] for i in range(int(len(clean_shakespeare)/100))][:3000]

In [4]:
import pickle as pkl

with open('./result/lstm_moliere.pkl', 'rb') as f:
    lstm_moliere = [cleaning(i)[:100] for i in pkl.load(f)]
    lstm_moliere = [i for i in lstm_moliere if len(i)==100]
    
with open('./result/lstm_shakespeare.pkl', 'rb') as f:
    lstm_shakespeare = [cleaning(i)[:100] for i in pkl.load(f)]
    lstm_shakespeare = [i for i in lstm_shakespeare if len(i)==100]


In [12]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

def create_input_output(moliere, shakespeare):
    dataset = moliere+shakespeare
    # create labels 
    Y = np.zeros([len(dataset), 2])

    Y[:len(moliere), 0] = 1
    Y[len(shakespeare):, 1] = 1

    # tokenize the text
    tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
    tokenizer.fit_on_texts(dataset)

    # create the data matrix, truncate it to 20 length
    x = tokenizer.texts_to_sequences(dataset)
    X = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=100)
    print(len(tokenizer.word_index))
    
    return X, Y

X, Y = create_input_output(dataset_moliere, dataset_shakespeare)
X_aug, Y_aug = create_input_output(lstm_moliere, lstm_shakespeare)

# train, validation and test splits
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.1)
X_train,  X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1*(1/0.9))

# X_train = np.concatenate([X_train, X_aug])
# Y_train = np.concatenate([Y_train, Y_aug])

38
38


In [13]:
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.models import Sequential

vocab_size = 38

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, restore_best_weights=True)

# create a simple LSTM model
model = Sequential()

model.add(Embedding(
    input_dim=vocab_size+1,
    output_dim=64,
    trainable=True))

model.add(LSTM(units=128, name='lstm_layer_1'))
model.add(Dropout(0.1))
model.add(Dense(units=256, activation='relu', name='dense_layer_1'))
model.add(Dropout(0.1))
model.add(Dense(units=2, name='output_layer', activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss = 'binary_crossentropy', 
      optimizer =  optimizer,
      metrics   =  ['acc'])
results = []
for i in range(10):
    model.fit(x=X_train, y = Y_train, validation_data=(X_valid, Y_valid), epochs=20, callbacks = [callback])
    result = model.evaluate(X_test, Y_test)
    results.append(result[1])

Epoch 1/20
150/150 [==============================] - 4s 15ms/step - loss: 0.6962 - acc: 0.5085 - val_loss: 0.6848 - val_acc: 0.5750
Epoch 2/20
150/150 [==============================] - 2s 13ms/step - loss: 0.6885 - acc: 0.5578 - val_loss: 0.6371 - val_acc: 0.6483
Epoch 3/20
150/150 [==============================] - 2s 13ms/step - loss: 0.5637 - acc: 0.7169 - val_loss: 0.4934 - val_acc: 0.7483
Epoch 4/20
150/150 [==============================] - 2s 13ms/step - loss: 0.4782 - acc: 0.7539 - val_loss: 0.4683 - val_acc: 0.7783
Epoch 5/20
150/150 [==============================] - 2s 13ms/step - loss: 0.4324 - acc: 0.8000 - val_loss: 0.4292 - val_acc: 0.7700
Epoch 6/20
150/150 [==============================] - 2s 12ms/step - loss: 0.3729 - acc: 0.8354 - val_loss: 0.4177 - val_acc: 0.8000
Epoch 7/20
150/150 [==============================] - 2s 12ms/step - loss: 0.3365 - acc: 0.8504 - val_loss: 0.3727 - val_acc: 0.8150
Epoch 8/20
150/150 [==============================] - 2s 12ms/step - 

With data augmentation:
    0.8936666548252106


In [14]:
sum(results)/10

0.8803333282470703